# Week 5 Notes

## Week 5 Lesson 1
## Recommendation Engines
9 - 20 June 2017

- Recommendation engines aims to match users to things (movies, songs, items, events, etc) they might enjoy but have not yet tried. The rating is produced by analysing other user/item ratings (and sometimes item characteristics) to provide personalised recommendations to users.
- There are two general approaches to the design:

### Content-based filtering
- In content-based filtering, items are mapped into a feature space, and recommendations depend on item characteristics.
- Content-based filtering begins by mapping each item into a feature space. Both users and items are represented by vectors in this space. 
- Item vectors measure the degree to which the item is described by each feature, and user vectors measure a user’s preferences for each feature.
- Ratings are generated by taking dot products of user & item vectors.
- Content-based filtering has some difficulties: 
 - Must map items into a feature space (manual work)
 -  Recommendations are limited in scope (items must be similar to each other)
 - Hard to create cross-content recommendations (eg books/music films...this would require comparing elements from different feature spaces)

<img src="img/collaborative_filtering.png" width=400 align=right>
### Collaborative filtering
- In contrast, the only data under consideration in collaborative filtering are user-item ratings, and recommendations depend on user preferences.
- Collaborative filtering refers to a family of methods for predicting ratings where instead of thinking about users and items in terms of a feature space, we are only interested in the existing user-item ratings themselves.
- In this case, our dataset is a ratings matrix whose columns correspond to items, and whose rows correspond to users. This will be the general form of the data we analyse for collaborative filtering. The method relies on previous user-item ratings (or feedback).


#### Cold-start problem
- Collaborative filtering is susceptible to the Cold Start problem.
- What happens if we don’t have any (or enough) reviews?
- Until users rate several items, we don’t know anything about their preferences.
- We can get around this by enhancing our recommendations using implicit feedback, which may include things like item browsing behaviour, search patterns, purchase history, etc. Or by using a hybrid model.

#### Jaccard Similarity:
- Defines similarity between two sets of objects.
<img src="img/jacard-similarity.png">

#### Measuring error
<img src="img/measuring-error.png" align=left>

#### Explicit vs Implicit Data
- Explicit data is when you ask the user to rate something, e.g. 1-5 star rating for a movie.
- Implicit data is when you observe a users behaviour and record.

#### Algorithms for Recommendations
- Alternating Least Squares (ALS) 
- Stochastic Gradient Descent (SGD) 
- Singular Value Decomposition (SVD) 
- Factorization Machine (FM) 
- Collaborative Less is More Filtering (CLiMF)

In [ ]:
!pip install scikit-surprise

In [ ]:
# Load Surprise
from surprise import SVD
from surprise import Dataset
from surprise import evaluate, print_perf
from surprise import Reader

In [ ]:
reader = Reader(line_format='user item rating timestamp', sep='\t')

data = Dataset.load_from_file(file_path = '../data/u.data', reader=reader)
data.split(n_folds=3)

algo = SVD()
perf = evaluate(algo, data, measures=['RMSE', 'MAE'])
print_perf(perf)

In [ ]:
# 3. Make individual recommendations
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)  # raw item id (as in the ratings file). They are **strings**!

# get a prediction for specific users and items.
pred = algo.predict(uid, iid, r_ui=2, verbose=True)

In [ ]:
#Other recommendation algorithms:
#random_pred.NormalPredictor    Algorithm predicting a random rating based on the distribution of the training set, which is assumed to be normal.
#baseline_only.BaselineOnly    Algorithm predicting the baseline estimate for given user and item.
#knns.KNNBasic    A basic collaborative filtering algorithm.
#knns.KNNWithMeans    A basic collaborative filtering algorithm, taking into account the mean ratings of each user.
#knns.KNNBaseline    A basic collaborative filtering algorithm taking into account a baseline rating.
#matrix_factorization.SVD    The famous SVD algorithm, as popularized by Simon Funk during the Netflix Prize.
#matrix_factorization.SVDpp    The SVD++ algorithm, an extension of SVD taking into account implicit ratings.
#matrix_factorization.NMF    A collaborative filtering algorithm based on Non-negative Matrix Factorization.
#slope_one.SlopeOne    A simple yet accurate collaborative filtering algorithm.
#co_clustering.CoClustering    A collaborative filtering algorithm based on co-clustering.

---

## Week 5 Lesson 2
## Databases + SQL in Python
10 - 22 June 2017

### Databases
- Databases are computer systems that manage storage and querying of data. Databases provide a way to organise data along with efficient methods to retrieve specific information.
- Typically, retrieval is performed using a query language, a mini programming syntax with a few basic operators for data transformation, the most common of which is SQL.
- Databases are the standard solution for data storage and are much more robust than text, CSV or json files. Most analyses involve pulling data to and from a resource and in most settings, that means using a database.
- Databases can come in many flavours, designed to serve for different use cases.
- Databases are ‘modelled’ to suit their intended purpose. 

#### Relational databases and schema
- A relational database is a database based tabular data and links between data entities or concepts.
- A relational database is organised into tables. Each table should correspond to one entity or concept.
- A table is made up rows and columns, similar to a Pandas dataframe or Excel spreadsheet.
- A table also has a schema which is a set of rules for what goes in each table. These specify what columns are contained in the table and what type those columns are (text, integers, floats, etc.).

#### Primary and foreign keys
- Each table typically has a primary key column. This column is a unique value per row and serves as the identifier for the row.
- A table can have many foreign keys as well. A foreign key is a column that contains values to link the table to the other tables.

#### Transactions
- A unit of work performed against a database is called a transaction. This term generally represents any change in database.
- ACID is a set of properties that guarantee that database transactions are processed reliably.
 - Atomicity "all or nothing": if one part of the transaction fails, the entire transaction fails, and the database state is left unchanged.
 - Consistency ensures that any transaction will bring the database from one valid state to another.
 - Isolation ensures that the concurrent execution of transactions results in a system state that would be obtained if transactions were executed serially, i.e., one after the other.
 - Durability ensures that once a transaction has been committed, it will remain so, even in the event of power loss, crashes, or errors.

### The sqlite3 package
- The command line utility can be useful for basic SQL tasks, but since we're using python for the rest of code it will often be easier to access sqlite directly from within python. We can use the python sqlite3 package for just this purpose.
- Open a connection to an SQLite database file. As before, if the file does not already exist it will automatically be created.

In [ ]:
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db) 
c = conn.cursor()

- The syntax to create a table is similar to the console, only now we use the execute method of the cursor object c that we just created:

In [ ]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

# Save (commit) the changes
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitutionusing the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

In [ ]:
last_sale = (None, 4000, 5, 22, 619000)
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)

# Remember to commit the changes
conn.commit()

- Notice that in this syntax we use the python None value, rather than NULL, to trigger SQLite to auto-increment the Primary Key.
- There is a related cursor method executemany() which takes an array of tuples and loops through them, substituting one tuple at a time.

In [ ]:
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

In [ ]:
c.execute('select * from houses').fetchall()

#### Adding data from a csv file
- Next let's load our housing.csv data into an array, and then INSERT those records into the database. In this example we'll use the numpy genfromtxt function to read the file and parse the contents.

In [ ]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists
data = genfromtxt('../data/housing-data.csv', dtype='i8', delimiter=',', skip_header=1).tolist()

# append a None value to beginning of each sub-list
for d in data:
    d.insert(0, None)

for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

- Remember that all elements in a numpy array must be the same data type, so if we want to 'add a None' to each row, we need to work around this. Lists can contain mixed types, so that is one approach.
- Still, in this case the value we're adding is the same for all records, so we could have simply used a 'None' in the INSERT statement directly.

#### Deleting data 

In [ ]:
# similar syntax as before
results = c.execute("SELECT * FROM houses WHERE bdrms = 4")

# here results is a cursor object - use fetchall() to extract a list
results.fetchall()

#### Pandas connector

- While databases provide many analytical capabilities, often it's useful to pull the data back into Python for more flexible programming. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.
- For example, if you want to aggregate nightly log-ins or sales to present a report or dashboard, this operation is likely not changing and operating on a large dataset. This can run very efficiently in a database rather than by connecting to it with Python.
- However, if we want to investigate login or sales data further and ask more interactive questions, then Python would be more practical.
- Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.
- SQLite creates portable SQL databases saved in a single file. These databases are stored in a very efficient manner and allow fast querying, making them ideal for small databases or databases that need to be moved across machines.

In [ ]:
import pandas as pd
from pandas.io import sql

### Writing data into a database

Data in Pandas can be loaded into a relational database. For the most part, Pandas can use column information to infer the schema for the table it creates.

In [ ]:
import pandas as pd

data = pd.read_csv('../data/housing-data.csv', low_memory=False)
data.head()

Data is moved to the database through the to_sql command, similar to the to_csv command.
to_sql takes as arguments:
- `name`, the table name to create
- `con`, a connection to a database
- `index`, whether to input the index column
- `schema`, if we want to write a custom schema for the new table
- `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

In [ ]:
data.to_sql('houses_pandas',
            con=conn,
            if_exists='replace',
            index=False)

In [ ]:
c.execute('select bdrms, avg(price) from houses_pandas group by bdrms').fetchall()

<a name="guided-practice"></a>


### SQL Syntax 


###### Select
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.
>
```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

##### Where

`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is some filter applied to the rows, where rows that match the condition will be in the output.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for when houses older than 60 years.

##### Aggregations

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate funtions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

##### Joins

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams. 
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

In [ ]:
import sqlite3
conn = sqlite3.connect('../data/enron.db') 
c = conn.cursor()
c.execute("SELECT name FROM sqlite_master WHERE type='table';").fetchall()
fields = c.execute("SELECT sql from sqlite_master WHERE type='table' and name='MessageBase';").fetchall()
print (''.join(fields[0]))
fields = c.execute("SELECT sql from sqlite_master WHERE type='table' and name='RecipientBase';").fetchall()
print (''.join(fields[0]))
fields = c.execute("SELECT sql from sqlite_master WHERE type='table' and name='EmployeeBase';").fetchall()
print (''.join(fields[0]))

results = c.execute("SELECT * FROM EmployeeBase LIMIT 5;").fetchall()
for row in results:
    print (row)

import pandas as pd
employees = pd.read_sql("SELECT * FROM EmployeeBase;", conn)
recipients = pd.read_sql("SELECT * FROM RecipientBase;", conn)
messages = pd.read_sql("SELECT * FROM MessageBase;", conn)

employees.describe()

#representing data
import scipy.special
from bokeh.layouts import gridplot
from bokeh.plotting import figure, output_notebook, show
p1 = figure(title="Distribution of Recipients",tools="save")
hist, edges = np.histogram(recipients.rno, density=False, bins=10)
x = np.arange(0,57)
p1.quad(top=hist, bottom=0, left=edges[:10], right=edges[1:], fill_color="#008080")
p1.legend.location = "center_right"
p1.xaxis.axis_label = 'No of Recipients'
p1.yaxis.axis_label = 'Messages'
output_notebook()
show(p1)

#Pandas merge
rm = pd.merge(recipients,messages,on="mid")
rme = pd.merge(rm,employees,left_on="from_eid",right_on="eid")

### Additional resources
- [sqlite3 home](http://www.sqlite.org)  
- [SQLite - Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  
- [SQL zoo](http://www.sqlzoo.net)  Great for learning syntax